# 探索 (search)

In [ ]:
import random
import math
from typing import NamedTuple, Any

一つのデータに対応した頂点

In [ ]:
class Node:
    def __init__(self, value, index:int):
        """
        初期化
        """
        self.value = value
        self.index = index # 頂点を区別するための番号
        # 左右の頂点　初期値はNone
        self.left:Node|None = None
        self.right:Node|None = None
    
    def __str__(self):
        """
        文字列化
        """
        leftStr = f'[{self.left.index}:{self.left.value}]' if self.left else '[]'
        rightStr = f'[{self.right.index}:{self.right.value}]' if self.right else '[]'
        return f'{leftStr} <- [{self.index}:{self.value}] -> {rightStr}'

探索用二分木

In [ ]:
class BinarySearchTree:
    def __init__(self, key = lambda x:x):
        """
        初期化

        Parameters
        ------
        key: リスト要素の扱い方
        """
        self.nodes:list[Node] = list()
        self.key = key
        self.index = 0

    def find_parent(self, value:Any) -> tuple[Node,str]:
        """
        親の頂点を返す

        returns
        -----
        p: 親のノード
        direction: 親からみた自身の位置(left / right)
        """
        node:Node|None = self.nodes[0]
        p:Node = node
        direction = ''
        while node:# nodeがNoneでない限り繰り返す
            p:Node = node
            if self.key(p.value) >= self.key(value):
                direction = 'left'
                node = p.left
            else:
                direction = 'right'
                node = p.right
        return p, direction
    
    def add(self, value:Any) -> None:
        """
        値を追加
        """
        node = Node(value,self.index)
        self.index += 1
        if self.nodes:# 頂点がすでにある場合
            #親となる頂点を探す
            parent, direction = self.find_parent(value)
            match direction:
                case 'left':
                    parent.left = node
                case 'right':
                    parent.right = node
                case _:
                    raise ValueError('Error in BinarySearchTree.add()')
        self.nodes.append(node)

    def search(self, value) -> tuple[int,Any]|None:
        """
        値を探す
        """
        node:Node|None = self.nodes[0]
        #ルートから順に二分木を辿る
        while node:
            if self.key(node.value) == self.key(value):
                return (node.index, node.value)
            elif self.key(node.value) > self.key(value):
                node = node.left
            else:
                node = node.right
        #見つからなかった場合
        return None

In [ ]:
def linearSearch(data:list, target, key=lambda x:x) -> int:
    for k, v in enumerate(data):
        if key(v) == key(target):
            return k
    return -1

In [ ]:
def testLinearSearch(n):
    input = [random.randint(-10, 10) for k in range(n)]
    k = linearSearch(input, 0)
    if k:# k がNoneで無い場合
        print(k, input[k])

In [ ]:
class P(NamedTuple):
    x:float
    y:float
    r:float

def drawTree(btree:BinarySearchTree, filename='binary.tex', rFac=1., d= 4.):
    """
    探索用二分木を描く: tikzpicture
    """
    r = (int(math.log2(len(btree.nodes))) + 1) * rFac
    pDict:dict[int,P]=dict()
    pDict[0]=P(0, 0, r)
    with open(filename, mode='w') as f:
        f.write('\\begin{tikzpicture}\n')
        for i,node in enumerate(btree.nodes):
            k = node.index
            p = pDict[k]
            if i == 0:
                f.write(f'\t\\node[my node] (n{node.index}) at ({p.x},{p.y}) {{({node.index},{node.value})}};\n')
            n:Node|None = node.left
            if n:
                c = P(p.x - p.r ,p.y - d / 2, p.r / 2)
                pDict[n.index] = c
                f.write(f'\t\\node[my node] (n{n.index}) at ({c.x},{c.y}) {{({n.index},{n.value})}};\n')
                f.write(f'\t\\draw[-] (n{node.index}) -- (n{n.index});\n')
            n:Node|None = node.right
            if n:
                c = P(p.x + p.r, p.y - d / 2, p.r / 2)
                pDict[n.index] = c
                f.write(f'\t\\node[my node] (n{n.index}) at ({c.x},{c.y}) {{({n.index},{n.value})}};\n')
                f.write(f'\t\\draw[-] (n{node.index}) -- (n{n.index});\n')
        f.write('\\end{tikzpicture}\n')


In [ ]:
def binarySearchSub(data:list, target, left:int, right:int, key=lambda x:x) -> tuple[int,Any]|None:
    """
    整列済みのリストから対象を探す、再帰的関数
    """
    if left >= right:
        return None
    if left == right - 1:
        if key(target) == key(data[left]):
            return left, data[left]
        return None
    m = (left + right) // 2
    if key(target) == key(data[m]):
        return m, data[m]
    if key(target) < key(data[m]):
        return binarySearchSub(data, target, left, m, key)
    return binarySearchSub(data, target, m, right, key)

def binarySearch(data:list[Any], target:Any, key=lambda x:x) -> tuple[int,Any]|None:
    """
    整列済みのリストから二分割探索する
    
    Parameters
    ------
    data: 対象となる整列済みリスト
    target: 探索対象
    key: リスト要素の扱い方
    
    returns
    ------
    発見した要素の位置、発見できなかった場合はNone
    """
    return binarySearchSub(data, target, 0, len(data), key)

In [ ]:
vList = [10,20,12,4,3,9,30,3,15]
btree = BinarySearchTree()
for v in vList:
    btree.add(v)

drawTree(btree)
for node in btree.nodes:
    print(node)

print('------')
target = 3
k = btree.search(target)
# k = binarySearch(sorted(vList), target)
if k:
    print(k)
else:
    print(f'target {target} is not found')